In [1]:
# Stated preference responses from person data
# Factors influencing walking, bike, and transit use

In [3]:
import numpy as np
import pandas as pd
# import load_data as data

In [113]:
trip = pd.read_csv(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Export\Release 3\General Release\Unzipped\2014-pr3-hhsurvey-trips.csv')

In [95]:
# Transit Access modes for all transit trips
expwt = 'expwt_final'

In [11]:
access_mode = trip.groupby("mode_acc")[expwt].count()
access_mode.to_clipboard()
access_mode

mode_acc
1.0    3003
2.0      47
3.0     162
4.0       4
5.0      63
6.0       2
7.0      15
Name: expwt_final, dtype: int64

In [171]:
# Access mode by county
pd.pivot_table(trip, values=expwt, index='mode_acc', columns='ocnty', aggfunc='sum').to_clipboard()

In [145]:
# Work trips only
work_trips = trip.query("d_purpose == 2")

In [173]:
pd.pivot_table(work_trips, values=expwt, index='mode_acc', columns='ocnty', aggfunc='count').to_clipboard()

In [35]:
trip.groupby(['transitsystem1','transitsystem2']).count()[expwt].to_clipboard()

In [62]:
trip.groupby('dcnty').sum()['sys1_exp']

dcnty
1    2.568328e+06
2    2.482631e+05
3    5.537981e+05
4    3.278869e+05
5    3.200257e+04
8    7.992686e+04
9             NaN
Name: sys1_exp, dtype: float64

In [61]:
trip['sys1_exp']=trip['transitsystem1']*trip[expwt]
trip['sys2_exp']=trip['transitsystem2']*trip[expwt]
trip['sys3_exp']=trip['transitsystem3']*trip[expwt]
trip['sys4_exp']=trip['transitsystem4']*trip[expwt]

In [134]:
(trip['transitsystem1']*trip[expwt]).count()

3317

In [146]:
for i in xrange(1,5):
    print i
    work_trips.ix[work_trips['transitsystem'+str(i)]>0, 'sys_flag'+str(i)] = 1
    work_trips.ix[work_trips['sys_flag'+str(i)] !=1, 'sys_flag'+str(i)] = 0
    
    work_trips['sys_exp'+str(i)] = work_trips['sys_flag'+str(i)]*work_trips[expwt]

1
2
3
4


In [ ]:
for i in xrange(1,5):
    print i
    trip.ix[trip['transitsystem'+str(i)]>0, 'sys_flag'+str(i)] = 1
    trip.ix[trip['sys_flag'+str(i)] !=1, 'sys_flag'+str(i)] = 0
    
    trip['sys_exp'+str(i)] = trip['sys_flag'+str(i)]*trip[expwt]

In [144]:
trip.groupby('dcnty').sum()['sys_flag4'].to_clipboard()

In [161]:
work_trips.groupby('dcnty').sum()['sys_flag3'].to_clipboard()

In [169]:
work_trips['sys_exp4'].sum()

0.0

In [19]:
# Transit access for work trips only
work_access_mode = work_trips.groupby("mode_acc")[expwt].sum()
work_access_mode.to_clipboard()


In [7]:
# Sound Transit trips only (should be about 40% motorized...)
stbus_trips = data.trip.query("transitsystem1 == 12")
sttrain_trips = data.trip.query("transitsystem1 == 13")

# Sound transit work trips only
stbus_trips_work = work_trips.query("transitsystem1 == 12")
sttrain_trips_work = work_trips.query("transitsystem1 == 13")

# All trips by Sound Transit
stbus_access = stbus_trips.groupby("mode_acc")[expwt].sum()
sttrain_trips = sttrain_trips.groupby("mode_acc")[expwt].sum()

# Home to Work trips only
stbus_access_work = stbus_trips_work.groupby("mode_acc")[expwt].sum()
sttrain_access_work = sttrain_trips_work.groupby("mode_acc")[expwt].sum()

sttrain_access_work.to_clipboard()

In [8]:
# Linked trips only
linked_trips = data.trip.query("num_trips_linked > 0")

In [9]:
# Access modes for linked trips
linked_acc = linked_trips.groupby("mode_acc")[expwt].sum()
linked_acc.to_clipboard()

In [10]:
# Mode split for linked trips
linked_mode = linked_trips.groupby("mode")[expwt].sum()
linked_mode.to_clipboard()

In [11]:
# How many transit trips have an access mode listed?
access_listed = data.trip.query("mode_acc > 0")
transit_trips = data.trip.query("mode == 8 or mode == 9 or mode == 10 or mode == 11")

In [12]:
print len(access_listed)
print len(transit_trips)

3301
3202


In [13]:
# Mode splits for access_listed
df = access_listed.groupby("mode")[expwt].sum()
df.to_clipboard()

In [14]:
# How many transit trips have a transit system listed?
df = transit_trips.groupby('transitsystem1')[expwt].sum()
df.to_clipboard()

In [15]:
data.trip.groupby('mode')[expwt].count()

mode
-99        30
 1      19480
 2      11384
 3       2781
 4        128
 5        123
 6        926
 7       8821
 8       2903
 9        168
 10       115
 11        15
 12       587
 13        87
 14        18
 15       169
 16       112
 17        32
dtype: int64

In [16]:
df = pd.pivot_table(data.trip, rows="mode", cols="mode_acc", aggfunc="count")[expwt]
df.to_clipboard()

In [17]:
# Access for KC Metro trips only


In [18]:
# Define KC subareas
# Reference: http://your.kingcounty.gov/budget/agr/agr03/03AGRCH4b.pdf
seashore_cities = ['SEATTLE', 'SHORELINE','LAKE FOREST PARK']
south_cities = ['BURIEN', 'NORMANDY PARK', 'SEATAC', 'TUKWILA', 'RENTON','DES MOINES', 'KENT', 'FEDERAL WAY',
                'AUBURN', 'ALGONA', 'MILTON', 'PACIFIC', 'COVINGTON', 'MAPLE VALLEY', 'BLACK DIAMOND']
east_cities = ['BOTHELL','WOODINVILLE', 'KENMORE', 'KIRKLAND', 'REDMOND','HUNTS POINT', 'YARROW POINT', 'CLYDE MILL',
               'MEDINA', 'BELLEVUE', 'BEAUX ARTS', 'SAMMAMISH', 'MERCER ISLAND', 'NEWCASTLE', 'ISSAQUAH']
all_kc_cities = seashore_cities + south_cities + east_cities

In [19]:
# Sort by trips ending in each subarea
seashore_df = data.trip.query('dcity in seashore_cities')
south_df = data.trip.query('dcity in south_cities')
east_df = data.trip.query('dcity in east_cities')

# Also consider trips to other places in King County
all_king_df = data.trip.query('dcnty == 1')
king_other_df = all_king_df.query('dcity not in all_kc_cities')

In [20]:
# Add a subarea field and merge together
seashore_df['subarea'] = 'Sea-Shore'
south_df['subarea'] = 'South'
east_df['subarea'] = 'East'

# All King County trips with subarea labels
kc_trips = pd.concat(objs=[seashore_df,south_df,east_df], axis=0, join='outer', 
               join_axes=None, ignore_index=False,
               keys=None, levels=None, names=None, verify_integrity=False)

In [22]:
df = pd.pivot_table(kc_trips, rows='mode_acc', cols='subarea', aggfunc='count')[expwt]
df.to_clipboard()

In [24]:
# commute trips only
work_trips_kc = kc_trips.query("d_purpose == 2")
df = pd.pivot_table(work_trips_kc, rows='mode_acc', cols='subarea', aggfunc='sum')[expwt]
df.to_clipboard()